In [ ]:
import gc
import random
from datetime import datetime

import pandas as pd
import numpy as np
import os
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import timm
import torchvision.models as models
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.1 MB/s eta 0:00:00


In [ ]:
from torchvision.models import EfficientNet_B2_Weights, EfficientNet_B3_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
# WeightsEnum.get_state_dict = get_state_dict

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [ ]:
!unzip -qq "/content/drive/MyDrive/AI부트캠프/프로젝트/CV프로젝트/open.zip"

In [ ]:
CFG = {
    'IMG_SIZE': 288,
    'EPOCHS': 15,
    'LEARNING_RATE': 1e-3,
    'BATCH_SIZE': 16,
    'SEED': 6
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
# running_colab = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
# if running_colab:
#     from google.colab import drive
#     drive.mount('/content/drive')
# if running_colab:
#     data_path = '/content/drive/MyDrive/Colab Notebooks/ai6th/data/optiver/'
# else:
#     data_path = '../../data/'

In [ ]:
df = pd.read_csv('train.csv')
df.loc[3896, 'artist'] = 'Titian'
df.loc[3986, 'artist'] = 'Alfred Sisley'
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch


In [ ]:
artists = df.groupby('artist')[['id']].count().rename(columns={'id':'count'}).reset_index()

In [ ]:
# Label Encoding
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=CFG['SEED'])

In [ ]:
train_df = train_df.sort_values(by=['id'])
train_df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
1,1,./train/0001.jpg,48
3,3,./train/0003.jpg,10
4,4,./train/0004.jpg,24
6,6,./train/0006.jpg,43


In [ ]:
val_df = val_df.sort_values(by=['id'])
val_df.head()

,id,img_path,artist
2,2,./train/0002.jpg,7
5,5,./train/0005.jpg,38
10,10,./train/0010.jpg,6
11,11,./train/0011.jpg,1
18,18,./train/0018.jpg,33


In [ ]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].values
    return df['img_path'].values, df['artist'].values

In [ ]:
train_img_paths, train_labels = get_data(train_df)
val_img_paths, val_labels = get_data(val_df)
all_img_paths, all_labels = get_data(df)

In [ ]:
from torchvision.transforms import ToTensor


class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms if transforms else ToTensor()

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']

        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
def resize_transform(size, state='train'):
    if state == 'train':
        transform = A.Compose([
                                A.HorizontalFlip(p=0.5),
                                A.VerticalFlip(p=0.5),
                                A.Rotate(p=0.5),
                                A.RandomRotate90(p=0.5),
                                A.RandomResizedCrop(height=size, width=size, scale=(0.3, 1.0)),
                                A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                ToTensorV2(),
                                ])
    else:
        transform = A.Compose([
                            A.Resize(size,size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

    return transform

In [ ]:
train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

In [ ]:
train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, worker_init_fn=seed_worker, generator=g, num_workers=4)

val_dataset = CustomDataset(val_img_paths, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=4)

In [ ]:
# class BaseModel(nn.Module):
#     def __init__(self, num_classes=len(le.classes_)):
#         super(BaseModel, self).__init__()
#         self.backbone = timm.create_model('efficientnet_b2', pretrained=True)
#         self.classifier = nn.Sequential(
#             nn.Linear(1000, num_classes)
#         )

#     def forward(self, x):
#         x = self.backbone(x)
#         x = self.classifier(x)
#         return x

In [ ]:
def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)

In [ ]:
class BaseModel(nn.Module): # 288
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        WeightsEnum.get_state_dict = get_state_dict

        models.efficientnet_b2(weights=EfficientNet_B2_Weights.IMAGENET1K_V1)
        self.backbone = models.efficientnet_b2(pretrained=True)

        self.classifier = nn.Sequential(
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x


In [ ]:
class EfficientNet_B3(nn.Module): # 320
    def __init__(self, num_classes=len(le.classes_)):
        super(EfficientNet_B3, self).__init__()
        WeightsEnum.get_state_dict = get_state_dict

        models.efficientnet_b3(weights=EfficientNet_B3_Weights.IMAGENET1K_V1)
        self.backbone = models.efficientnet_b3(pretrained=True)

        self.classifier = nn.Sequential(
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
class tiny_vit(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(tiny_vit, self).__init__()
        self.backbone = timm.create_model('tiny_vit_21m_384.dist_in22k_ft_in1k', pretrained=True)
        # backbone 모델을 tiny_vit384로 설정
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
def clear_mem():
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
def train(model, optimizer, criterion, train_loader, device):
    model.train()
    train_loss = []
    for img, label in iter(train_loader):
        img, label = img.float().to(device), label.long().to(device)

        optimizer.zero_grad()

        model_pred = model(img)

        loss = criterion(model_pred, label)

        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    return np.mean(train_loss)

In [ ]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()

    model_preds = []
    true_labels = []

    val_loss = []

    with torch.no_grad():
        for img, label in iter(test_loader):
            img, label = img.float().to(device), label.long().to(device)

            model_pred = model(img)

            loss = criterion(model_pred, label)

            val_loss.append(loss.item())

            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()

    val_f1 = competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [ ]:
# time_now = datetime.now()
# run_id = time_now.strftime("%Y%m%d%H%M%S")
# os.makedirs(os.path.join('/content/drive/MyDrive/AI부트캠프/프로젝트/CV프로젝트', f'./runs/{run_id}'), exist_ok=True)
# print(f'{run_id=}')

In [ ]:
def train_epoch(model, optimizer, train_loader, test_loader, scheduler, device, model_name):
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    early_stopping = EarlyStopping(patience=10, verbose=True)

    best_score = 0

    for epoch in range(1,CFG["EPOCHS"]+1):
        tr_loss = train(model, optimizer, criterion, train_loader, device)
        val_loss, val_score = validation(model, criterion, test_loader, device)

        if scheduler is not None:
            scheduler.step()

        if best_score < val_score:
            print(f'**Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
            best_score = val_score
            torch.save(model, os.path.join('/content/drive/MyDrive/AI부트캠프/프로젝트/CV프로젝트', f'{model_name}.pt'))
            print("weight file was updated")
        else:
            print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
        clear_mem()
        if early_stopping(val_score):
            print(f'Epoch [{epoch}], early stopping')
            break

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            print(f'Best F1 score from now: {self.best_score}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

        return self.early_stop

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

train_epoch(model, optimizer, train_loader, val_loader, scheduler, device, 'basemodel')

**Epoch [1], Train Loss : [2.67889] Val Loss : [2.17926] Val F1 Score : [0.31681]
weight file was updated
**Epoch [2], Train Loss : [1.88552] Val Loss : [1.61134] Val F1 Score : [0.45065]
weight file was updated
**Epoch [3], Train Loss : [1.54804] Val Loss : [1.47037] Val F1 Score : [0.53120]
weight file was updated
Epoch [4], Train Loss : [1.26781] Val Loss : [1.60447] Val F1 Score : [0.50148]
EarlyStopping counter: 1 out of 10
Best F1 score from now: 0.5311956880104849
**Epoch [5], Train Loss : [1.06125] Val Loss : [1.37323] Val F1 Score : [0.58720]
weight file was updated
Epoch [6], Train Loss : [0.90625] Val Loss : [1.36932] Val F1 Score : [0.58386]
EarlyStopping counter: 1 out of 10
Best F1 score from now: 0.5871952346706822
Epoch [7], Train Loss : [0.80089] Val Loss : [1.41790] Val F1 Score : [0.57702]
EarlyStopping counter: 2 out of 10
Best F1 score from now: 0.5871952346706822
**Epoch [8], Train Loss : [0.66338] Val Loss : [1.49590] Val F1 Score : [0.60161]
weight file was upda

In [ ]:
train_dataset_2 = CustomDataset(train_img_paths, train_labels, resize_transform(320,'train'))
train_loader_2 = DataLoader(train_dataset_2, batch_size = 32, shuffle=True, num_workers=4)

val_dataset_2 = CustomDataset(val_img_paths, val_labels, resize_transform(320,'test'))
val_loader_2 = DataLoader(val_dataset_2, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
model2 = EfficientNet_B3()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

train_epoch(model, optimizer, train_loader_2, val_loader_2, scheduler, device, 'efficientnetb3')

Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-cf984f9c.pth
100%|██████████| 47.2M/47.2M [00:03<00:00, 12.5MB/s]


**Epoch [1], Train Loss : [1.41541] Val Loss : [1.10998] Val F1 Score : [0.65953]
weight file was updated
Epoch [2], Train Loss : [1.08512] Val Loss : [1.10913] Val F1 Score : [0.64496]
EarlyStopping counter: 1 out of 10
Best F1 score from now: 0.6595283769371761
**Epoch [3], Train Loss : [0.97411] Val Loss : [1.14550] Val F1 Score : [0.67519]
weight file was updated
Epoch [4], Train Loss : [0.87871] Val Loss : [1.09215] Val F1 Score : [0.64296]
EarlyStopping counter: 1 out of 10
Best F1 score from now: 0.675188131239813
**Epoch [5], Train Loss : [0.84828] Val Loss : [0.98852] Val F1 Score : [0.69469]
weight file was updated
Epoch [6], Train Loss : [0.75552] Val Loss : [1.06512] Val F1 Score : [0.68131]
EarlyStopping counter: 1 out of 10
Best F1 score from now: 0.6946897975252975
**Epoch [7], Train Loss : [0.72629] Val Loss : [1.03689] Val F1 Score : [0.69947]
weight file was updated
Epoch [8], Train Loss : [0.69789] Val Loss : [1.09807] Val F1 Score : [0.68041]
EarlyStopping counter: 

In [ ]:
train_dataset_3 = CustomDataset(train_img_paths, train_labels, resize_transform(800,'train'))
train_loader_3 = DataLoader(train_dataset_3, batch_size = 32, shuffle=True, num_workers=4)

val_dataset_3 = CustomDataset(val_img_paths, val_labels, resize_transform(800,'test'))
val_loader_3 = DataLoader(val_dataset_3, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
model3 = tiny_vit()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

train_epoch(model, optimizer, train_loader, val_loader, scheduler, device, 'tinyvit')

model.safetensors:   0%|          | 0.00/85.0M [00:00<?, ?B/s]

**Epoch [1], Train Loss : [0.51998] Val Loss : [1.32528] Val F1 Score : [0.67620]
weight file was updated
**Epoch [2], Train Loss : [0.41334] Val Loss : [1.52974] Val F1 Score : [0.67936]
weight file was updated
Epoch [3], Train Loss : [0.36873] Val Loss : [1.56789] Val F1 Score : [0.64860]
EarlyStopping counter: 1 out of 10
Best F1 score from now: 0.6793599614274184
Epoch [4], Train Loss : [0.32314] Val Loss : [1.47971] Val F1 Score : [0.66685]
EarlyStopping counter: 2 out of 10
Best F1 score from now: 0.6793599614274184
Epoch [5], Train Loss : [0.29343] Val Loss : [1.47334] Val F1 Score : [0.66470]
EarlyStopping counter: 3 out of 10
Best F1 score from now: 0.6793599614274184
**Epoch [6], Train Loss : [0.27580] Val Loss : [1.29074] Val F1 Score : [0.69760]
weight file was updated
Epoch [7], Train Loss : [0.26457] Val Loss : [1.50434] Val F1 Score : [0.65126]
EarlyStopping counter: 1 out of 10
Best F1 score from now: 0.6975952575224342
Epoch [8], Train Loss : [0.25106] Val Loss : [1.41

In [ ]:
test_df = pd.read_csv('./test.csv')
test_df.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [ ]:
test_img_paths = get_data(test_df, infer=True)

In [ ]:
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()

    model_preds = []

    with torch.no_grad():
        for img in test_loader:
            img = img.float().to(device)

            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()

    print('Done.')
    return model_preds

In [ ]:
checkpoint = os.path.join(data_path, f'runs/{run_id}/best_model.pt')
print(f'CHECKPOINT LOADED: {checkpoint}')
infer_model = torch.load(checkpoint)
infer_model.to(device)
infer_model.eval()

CHECKPOINT LOADED: ../data\runs/20231213133958/best_model.pt


BaseModel(
  (backbone): EfficientNet(
    (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNormAct2d(
            48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(

In [ ]:
preds = inference(infer_model, test_loader, device)

Done.


In [ ]:
preds = le.inverse_transform(preds)

In [ ]:
submit = pd.read_csv(os.path.join(data_path, './sample_submission.csv'))

In [ ]:
submit['artist'] = preds

In [ ]:
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Salvador Dali
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Henri Matisse


In [ ]:
submit.to_csv(os.path.join(data_path, './submit.csv'), index=False)